In [1]:
from scipy.spatial import cKDTree
import pandas as pd
import lasio 
import numpy as np
import os
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline

C:\Users\Иван\AppData\Local\Temp\ipykernel_17892\3908856017.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
def parse_traj_data(file_path: str) -> pd.DataFrame:
    with open(file_path, 'r') as file:
        lines = file.readlines()
        data_start = 0
        for i, line in enumerate(lines):
            if line.strip().startswith('#====='):
                data_start = i + 1
                break
            
        column_names = lines[data_start].split()
        df = pd.DataFrame([x.split() for x in lines[data_start + 2:]], columns=column_names)
        df = df.apply(pd.to_numeric, errors='ignore')

    return df


def find_closest_indices_xyz(coords: np.ndarray, target_coords: np.ndarray) -> np.ndarray:
    tree = cKDTree(coords)
    distances, indices = tree.query(target_coords)
    return indices

def interpolate_well_trajectory(df, new_step):
    md = df['MD'].values
    x = df['X'].values
    y = df['Y'].values
    z = df['Z'].values

    spline_x = CubicSpline(md, x)
    spline_y = CubicSpline(md, y)
    spline_z = CubicSpline(md, z)

    md_new = np.arange(md.min(), md.max(), new_step)

    x_new = spline_x(md_new)
    y_new = spline_y(md_new)
    z_new = spline_z(md_new)

    df_new = pd.DataFrame({'MD': md_new,
                           'X': x_new,
                           'Y': y_new,
                           'Z': z_new
                          })
    return df_new




In [9]:
traj_df2 = parse_traj_data(file_path)
traj_df2

C:\Users\Иван\AppData\Local\Temp\ipykernel_17892\3763998010.py:12: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df = df.apply(pd.to_numeric, errors='ignore')


,MD,X,Y,Z,TVD,DX,DY,AZIM_TN,INCL,DLS,AZIM_GN
0,0.000000,389302.73000,7.523500e+06,53.000000,0.000000,2.000000e-09,2.800000e-09,357.567996,0.000000,0.000000,0.000000
1,83.000000,389302.73000,7.523500e+06,-30.000000,83.000000,2.000000e-09,2.800000e-09,0.000000,0.000000,0.000000,2.432004
2,113.000000,389302.73000,7.523500e+06,-60.000000,113.000000,2.000000e-09,2.800000e-09,0.000000,0.000000,0.000000,2.432004
3,143.000000,389302.73000,7.523500e+06,-90.000000,143.000000,2.000000e-09,2.800000e-09,0.000000,0.000000,0.000000,2.432004
4,173.000000,389302.73000,7.523500e+06,-120.000000,173.000000,2.000000e-09,2.800000e-09,0.000000,0.000000,0.000000,2.432004
...,...,...,...,...,...,...,...,...,...,...,...
386,3390.409987,389843.24647,7.521875e+06,-1854.664596,1907.664596,5.405165e+02,-1.625327e+03,156.068235,89.710073,0.000004,158.500240
387,3400.412452,389846.91144,7.521866e+06,-1854.715211,1907.715211,5.441814e+02,-1.634631e+03,156.068237,89.710073,0.000004,158.500241
388,3410.415006,389850.57640,7.521857e+06,-1854.765825,1907.765825,5.478464e+02,-1.643935e+03,156.068343,89.710074,0.000319,158.500348
389,3420.417564,389854.24137,7.521847e+06,-1854.816439,1907.816439,5.515114e+02,-1.653239e+03,156.067844,89.710067,0.001498,158.499848


In [3]:
file_path = 'data/Скважина_2'
traj_df = parse_traj_data(file_path)
traj_df.Z = traj_df.Z.to_numpy() * -1
traj_df = interpolate_well_trajectory(traj_df, new_step=0.1) # интерполирует траекторию под нужный шаг дискретизации
traj_df

C:\Users\Иван\AppData\Local\Temp\ipykernel_17892\3763998010.py:12: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df = df.apply(pd.to_numeric, errors='ignore')


,MD,X,Y,Z
0,0.0,389302.730000,7.523500e+06,-53.000000
1,0.1,389302.730000,7.523500e+06,-52.900000
2,0.2,389302.730000,7.523500e+06,-52.800000
3,0.3,389302.730000,7.523500e+06,-52.700000
4,0.4,389302.730000,7.523500e+06,-52.600000
...,...,...,...,...
34229,3422.9,389855.150966,7.521845e+06,1854.829001
34230,3423.0,389855.187608,7.521845e+06,1854.829507
34231,3423.1,389855.224249,7.521845e+06,1854.830013
34232,3423.2,389855.260891,7.521845e+06,1854.830519


In [4]:
grid_df = pd.read_csv('data/grid.csv')
grid_df

,X_UTME,Y_UTMN,Z_TVDSS,GAMMARAY
0,386462.90625,7524920.0,1300.011246,73.9481
1,386462.90625,7524920.0,1300.084991,73.9481
2,386462.90625,7524920.0,1300.257492,73.9481
3,386462.90625,7524920.0,1300.526245,73.9481
4,386462.90625,7524920.0,1300.880005,73.9481
...,...,...,...,...
34046912,394962.90625,7524870.0,2121.257507,75.9989
34046913,394962.90625,7524870.0,2121.507507,75.9554
34046914,394962.90625,7524870.0,2121.757507,75.9583
34046915,394962.90625,7524870.0,2121.963776,75.9895


In [5]:
indices = find_closest_indices_xyz(target_coords=traj_df[['X', 'Y', 'Z']].to_numpy(), 
                                   coords=grid_df[['X_UTME', 'Y_UTMN', 'Z_TVDSS']].to_numpy()
                                   )
                                   
curve = grid_df.GAMMARAY.to_numpy()[indices]

In [6]:
las = lasio.LASFile()
las.insert_curve(0, "DEPT", traj_df['MD'], unit="m", descr="Depth")
las.insert_curve(1, "GR", curve, unit="API", descr="Gamma Ray")
las.write("gamma_ray_curve.las", version=2.0)


In [10]:
indices

array([11336443, 11336443, 11336443, ..., 14318252, 14318252, 14318252],
      dtype=int64)

In [14]:
grid_df.GAMMARAY.to_numpy()

array([73.9481, 73.9481, 73.9481, ..., 75.9583, 75.9895, 76.0056])